# Home Credit

今回はゴリゴリ特徴を増やし続けてどうなるかを検証して見ました。  
EDAを中心にやっているカーネルは別途でGitHubに上がっているので、ご覧頂ければ幸いです。  
ローカルで実行するとメモリ１００GB以上取られてクラッシュするのでオススメしません。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.preprocessing import LabelEncoder
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import lightgbm as lgb

ラベルエンコーダーを関数化

In [2]:
le = LabelEncoder()
def les(data):
    
    for col in data:
        if data[col].dtype == "object":
            if len(list(data[col].unique())) == 2:
                le.fit(data[col])
            
                data[col] = le.transform(data[col])
                
    data = pd.get_dummies(data)
    return data

分かりにくいですが、groupbysのなかでcolumn_change関数を実装しています。  
トランザクションデータはユーザーが取引した記録なので、重複していることに注目しgroupe byを使い特徴を増やしています。  


In [3]:
def column_change(data,word,ID):
    data_rename = data
    col_list = []
    for col in  data.columns:
        if ID == col:
            col_list.append(col)
        else:
            col_list.append(col+"_"+word)
            
        print(word)
        print(col)
    data_rename.columns = col_list
    
#     data_rename[ID] = data_rename[ID+"_"+word]
#     data_rename = data_rename.drop(ID+"_"+word, axis = 1)
    return  data_rename

In [4]:
#１groupbyするデータ 2merge先データ　３　groupbyの特徴名
def groupbys(data1,data2,name):
    byword = ["mean","sum","max","min","std","count","median","quantile","slew","kurt","var"]
        
    data_mean= data1.groupby(name).mean()
    data_mean = column_change(data_mean,"mean",name)
    data2 =  pd.merge(data2, data_mean,on=name,how='left')
    
    data_sum = data1.groupby(name).sum()
    data_sum = column_change(data_sum,"sum",name)
    data2 =  pd.merge(data2, data_sum,on=name,how='left')
    
    data_max = data1.groupby(name).max()
    data_max = column_change(data_max,"max",name)
    data2 =  pd.merge(data2, data_max,on=name,how='left')
    
    data_min = data1.groupby(name).min()
    data_min = column_change(data_min,"min",name)
    data2 =  pd.merge(data2, data_min,on=name,how='left')
    
    data_std = data1.groupby(name).std()
    data_std = column_change(data_std,"std",name)
    data2 =  pd.merge(data2, data_std,on=name,how='left')
    
    data_count = data1.groupby(name).count()
    data_count = column_change(data_count,"count",name)
    data2 =  pd.merge(data2, data_count,on=name,how='left')
    
    data_median = data1.groupby(name).median()
    data_median = column_change(data_median,"median",name)
    data2 =  pd.merge(data2, data_median,on=name,how='left')
    
#     data_quantle = data1.groupby(name).quantile()
#     data_quantle = column_change(data_quantle,"quantle",name)
#     data2 =  pd.merge(data2, data_quantle,on=name,how='left')
    
#     data_skew = data1.groupby(name).skew()
#     data_skew = column_change(data_skew,"skew",name)
#     data2 =  pd.merge(data2, data_skew,on=name,how='left')
    
#     data_new = data1.groupby(name).kurt()
#     data2 =  pd.merge(data2, data_new,on=name,how='left')
    
    data_var = data1.groupby(name).var()
    data_var = column_change(data_var,"var",name)
    data2 =  pd.merge(data2, data_var,on=name,how='left')
    
    return data2

In [5]:
bbalance = pd.read_csv("input/bureau_balance.csv")

ここから欠損の異常値や欠損の処理とマージを行なっています。

In [6]:
bbalance["STATUS"] = bbalance["STATUS"].replace("X",10)
bbalance["STATUS"] = bbalance["STATUS"].replace("C",0)
bbalance["STATUS"] = bbalance["STATUS"].map(int)
bbalance["STATUS"] = bbalance["STATUS"].replace(10,np.NaN)

In [8]:
bbalance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,0.0
1,5715448,-1,0.0
2,5715448,-2,0.0
3,5715448,-3,0.0
4,5715448,-4,0.0


In [9]:
bureau['DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_ENDDATE'][bureau['DAYS_CREDIT_ENDDATE'] < -40000] = np.nan
bureau['DAYS_CREDIT_UPDATE']= bureau['DAYS_CREDIT_UPDATE'][bureau['DAYS_CREDIT_UPDATE'] < -40000] = np.nan
bureau['DAYS_ENDDATE_FACT']=bureau['DAYS_ENDDATE_FACT'][bureau['DAYS_ENDDATE_FACT'] < -40000] = np.nan

In [10]:
bureau['bureau_credit_active_binary'] = (bureau['CREDIT_ACTIVE'] != 'Closed').astype(int)
bureau['bureau_credit_enddate_binary'] = (bureau['DAYS_CREDIT_ENDDATE'] > 0).astype(int)

In [11]:
bureau = les(bureau)

In [13]:
bureau = groupbys(bbalance,bureau,"SK_ID_BUREAU")

mean
MONTHS_BALANCE
mean
STATUS
sum
MONTHS_BALANCE
sum
STATUS
max
MONTHS_BALANCE
max
STATUS
min
MONTHS_BALANCE
min
STATUS
std
MONTHS_BALANCE
std
STATUS
count
MONTHS_BALANCE
count
STATUS
median
MONTHS_BALANCE
median
STATUS
var
MONTHS_BALANCE
var
STATUS


In [14]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_min,STATUS_min,MONTHS_BALANCE_std,STATUS_std,MONTHS_BALANCE_count,STATUS_count,MONTHS_BALANCE_median,STATUS_median,MONTHS_BALANCE_var,STATUS_var
0,215354,5714462,-497,0,NaN,NaN,NaN,0,91323.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,-208,0,NaN,NaN,NaN,0,225000.0,171342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,NaN,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,NaN,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
train = pd.read_csv("input/application_train.csv")
test = pd.read_csv("input/application_test.csv")

In [16]:
train_test = train.append(test)

In [20]:
train_test = groupbys(bureau,train_test,"SK_ID_CURR") 

mean
SK_ID_BUREAU
mean
DAYS_CREDIT
mean
CREDIT_DAY_OVERDUE
mean
DAYS_CREDIT_ENDDATE
mean
DAYS_ENDDATE_FACT
mean
AMT_CREDIT_MAX_OVERDUE
mean
CNT_CREDIT_PROLONG
mean
AMT_CREDIT_SUM
mean
AMT_CREDIT_SUM_DEBT
mean
AMT_CREDIT_SUM_LIMIT
mean
AMT_CREDIT_SUM_OVERDUE
mean
DAYS_CREDIT_UPDATE
mean
AMT_ANNUITY
mean
bureau_credit_active_binary
mean
bureau_credit_enddate_binary
mean
CREDIT_ACTIVE_Active
mean
CREDIT_ACTIVE_Bad debt
mean
CREDIT_ACTIVE_Closed
mean
CREDIT_ACTIVE_Sold
mean
CREDIT_CURRENCY_currency 1
mean
CREDIT_CURRENCY_currency 2
mean
CREDIT_CURRENCY_currency 3
mean
CREDIT_CURRENCY_currency 4
mean
CREDIT_TYPE_Another type of loan
mean
CREDIT_TYPE_Car loan
mean
CREDIT_TYPE_Cash loan (non-earmarked)
mean
CREDIT_TYPE_Consumer credit
mean
CREDIT_TYPE_Credit card
mean
CREDIT_TYPE_Interbank credit
mean
CREDIT_TYPE_Loan for business development
mean
CREDIT_TYPE_Loan for purchase of shares (margin lending)
mean
CREDIT_TYPE_Loan for the purchase of equipment
mean
CREDIT_TYPE_Loan for working capi

In [22]:
train_test.shape

(356255, 554)

In [23]:
train_test.to_csv("new_data/train_test.csv")

In [24]:
pos = pd.read_csv("input/POS_CASH_balance.csv")

In [25]:
pos =les(pos)

In [26]:
train_test = groupbys(pos,train_test,"SK_ID_CURR") 

mean
SK_ID_PREV
mean
MONTHS_BALANCE
mean
CNT_INSTALMENT
mean
CNT_INSTALMENT_FUTURE
mean
SK_DPD
mean
SK_DPD_DEF
mean
NAME_CONTRACT_STATUS_Active
mean
NAME_CONTRACT_STATUS_Amortized debt
mean
NAME_CONTRACT_STATUS_Approved
mean
NAME_CONTRACT_STATUS_Canceled
mean
NAME_CONTRACT_STATUS_Completed
mean
NAME_CONTRACT_STATUS_Demand
mean
NAME_CONTRACT_STATUS_Returned to the store
mean
NAME_CONTRACT_STATUS_Signed
mean
NAME_CONTRACT_STATUS_XNA
sum
SK_ID_PREV
sum
MONTHS_BALANCE
sum
CNT_INSTALMENT
sum
CNT_INSTALMENT_FUTURE
sum
SK_DPD
sum
SK_DPD_DEF
sum
NAME_CONTRACT_STATUS_Active
sum
NAME_CONTRACT_STATUS_Amortized debt
sum
NAME_CONTRACT_STATUS_Approved
sum
NAME_CONTRACT_STATUS_Canceled
sum
NAME_CONTRACT_STATUS_Completed
sum
NAME_CONTRACT_STATUS_Demand
sum
NAME_CONTRACT_STATUS_Returned to the store
sum
NAME_CONTRACT_STATUS_Signed
sum
NAME_CONTRACT_STATUS_XNA
max
SK_ID_PREV
max
MONTHS_BALANCE
max
CNT_INSTALMENT
max
CNT_INSTALMENT_FUTURE
max
SK_DPD
max
SK_DPD_DEF
max
NAME_CONTRACT_STATUS_Active
max
NAME

In [27]:
train_test.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,SK_DPD_DEF_var,NAME_CONTRACT_STATUS_Active_var,NAME_CONTRACT_STATUS_Amortized debt_var,NAME_CONTRACT_STATUS_Approved_var,NAME_CONTRACT_STATUS_Canceled_var,NAME_CONTRACT_STATUS_Completed_var,NAME_CONTRACT_STATUS_Demand_var,NAME_CONTRACT_STATUS_Returned to the store_var,NAME_CONTRACT_STATUS_Signed_var,NAME_CONTRACT_STATUS_XNA_var
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.068783,0.0,0.0,0.0,0.068783,0.0,0.000000,0.000000,0.0
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.250000,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.128571,0.0,0.0,0.0,0.090476,0.0,0.047619,0.000000,0.0
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.057809,0.0,0.0,0.0,0.044056,0.0,0.000000,0.015152,0.0


In [28]:
insta = pd.read_csv("input/installments_payments.csv")

In [29]:
insta["PAYMENT_PERC"] = insta["AMT_PAYMENT"] / insta["AMT_INSTALMENT"]
insta["PAYMENT_DIFF"] = insta["AMT_INSTALMENT"] - insta["AMT_PAYMENT"]

insta["DPD"] = insta["DAYS_ENTRY_PAYMENT"] - insta["DAYS_INSTALMENT"]
insta["DBD"] = insta["DAYS_INSTALMENT"] - insta["DAYS_ENTRY_PAYMENT"]
insta["DPD"] = insta["DPD"].apply(lambda x: x if x > 0 else 0)
insta["DBD"] = insta["DBD"].apply(lambda x: x if x > 0 else 0)

In [33]:
insta.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,PAYMENT_PERC,PAYMENT_DIFF,DPD,DBD
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360,1.000000,0.000,0.0,7.0
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525,1.000000,0.000,0.0,0.0
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000,1.000000,0.000,0.0,0.0
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130,1.000000,0.000,0.0,8.0
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585,0.997942,4.455,17.0,0.0


In [34]:
train_test = groupbys(insta,train_test,"SK_ID_CURR")

mean
SK_ID_PREV
mean
NUM_INSTALMENT_VERSION
mean
NUM_INSTALMENT_NUMBER
mean
DAYS_INSTALMENT
mean
DAYS_ENTRY_PAYMENT
mean
AMT_INSTALMENT
mean
AMT_PAYMENT
mean
PAYMENT_PERC
mean
PAYMENT_DIFF
mean
DPD
mean
DBD
sum
SK_ID_PREV
sum
NUM_INSTALMENT_VERSION
sum
NUM_INSTALMENT_NUMBER
sum
DAYS_INSTALMENT
sum
DAYS_ENTRY_PAYMENT
sum
AMT_INSTALMENT
sum
AMT_PAYMENT
sum
PAYMENT_PERC
sum
PAYMENT_DIFF
sum
DPD
sum
DBD
max
SK_ID_PREV
max
NUM_INSTALMENT_VERSION
max
NUM_INSTALMENT_NUMBER
max
DAYS_INSTALMENT
max
DAYS_ENTRY_PAYMENT
max
AMT_INSTALMENT
max
AMT_PAYMENT
max
PAYMENT_PERC
max
PAYMENT_DIFF
max
DPD
max
DBD
min
SK_ID_PREV
min
NUM_INSTALMENT_VERSION
min
NUM_INSTALMENT_NUMBER
min
DAYS_INSTALMENT
min
DAYS_ENTRY_PAYMENT
min
AMT_INSTALMENT
min
AMT_PAYMENT
min
PAYMENT_PERC
min
PAYMENT_DIFF
min
DPD
min
DBD
std
SK_ID_PREV
std
NUM_INSTALMENT_VERSION
std
NUM_INSTALMENT_NUMBER
std
DAYS_INSTALMENT
std
DAYS_ENTRY_PAYMENT
std
AMT_INSTALMENT
std
AMT_PAYMENT
std
PAYMENT_PERC
std
PAYMENT_DIFF
std
DPD
std
DBD
count
SK_

In [35]:
train_test.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,NUM_INSTALMENT_VERSION_var,NUM_INSTALMENT_NUMBER_var,DAYS_INSTALMENT_var,DAYS_ENTRY_PAYMENT_var,AMT_INSTALMENT_var,AMT_PAYMENT_var,PAYMENT_PERC_var,PAYMENT_DIFF_var,DPD_var,DBD_var
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.052632,31.666667,28500.000000,29604.257310,1.011641e+08,1.011641e+08,0.000000,0.000000e+00,0.000000,24.257310
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.040000,9.826667,573735.890000,573541.810000,1.221966e+10,1.221966e+10,0.000000,0.000000e+00,0.000000,13.890000
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.333333,1.000000,900.000000,1157.333333,9.071372e+06,9.071372e+06,0.000000,0.000000e+00,0.000000,17.333333
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.116667,8.395833,31817.000000,39222.116667,2.825681e+10,2.825681e+10,0.000000,0.000000e+00,0.000000,645.050000
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.141026,16.751748,465297.965501,461933.971096,6.166821e+07,6.477128e+07,0.043995,8.084830e+06,5.305594,49.660839


In [36]:
credit = pd.read_csv("input/credit_card_balance.csv")

In [37]:
credit['AMT_DRAWINGS_ATM_CURRENT'] = credit['AMT_DRAWINGS_ATM_CURRENT'][credit['AMT_DRAWINGS_ATM_CURRENT'] < 0] = np.nan
credit['AMT_DRAWINGS_CURRENT'] = credit['AMT_DRAWINGS_CURRENT'][credit['AMT_DRAWINGS_CURRENT'] < 0] = np.nan

In [52]:
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,CNT_INSTALMENT_MATURE_CUM,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,2562384,378907,-6,56.970,135000,NaN,NaN,0.0,877.5,1700.325,...,35.0,0,0,1,0,0,0,0,0,0
1,2582071,363914,-1,63975.555,45000,NaN,NaN,0.0,0.0,2250.000,...,69.0,0,0,1,0,0,0,0,0,0
2,1740877,371185,-7,31815.225,450000,NaN,NaN,0.0,0.0,2250.000,...,30.0,0,0,1,0,0,0,0,0,0
3,1389973,337855,-4,236572.110,225000,NaN,NaN,0.0,0.0,11795.760,...,10.0,0,0,1,0,0,0,0,0,0
4,1891521,126868,-1,453919.455,450000,NaN,NaN,0.0,11547.0,22924.890,...,101.0,0,0,1,0,0,0,0,0,0


In [54]:
credit = les(credit)

In [55]:
train_test = groupbys(credit,train_test,"SK_ID_CURR")

mean
SK_ID_PREV
mean
MONTHS_BALANCE
mean
AMT_BALANCE
mean
AMT_CREDIT_LIMIT_ACTUAL
mean
AMT_DRAWINGS_ATM_CURRENT
mean
AMT_DRAWINGS_CURRENT
mean
AMT_DRAWINGS_OTHER_CURRENT
mean
AMT_DRAWINGS_POS_CURRENT
mean
AMT_INST_MIN_REGULARITY
mean
AMT_PAYMENT_CURRENT
mean
AMT_PAYMENT_TOTAL_CURRENT
mean
AMT_RECEIVABLE_PRINCIPAL
mean
AMT_RECIVABLE
mean
AMT_TOTAL_RECEIVABLE
mean
CNT_DRAWINGS_ATM_CURRENT
mean
CNT_DRAWINGS_CURRENT
mean
CNT_DRAWINGS_OTHER_CURRENT
mean
CNT_DRAWINGS_POS_CURRENT
mean
CNT_INSTALMENT_MATURE_CUM
mean
SK_DPD
mean
SK_DPD_DEF
mean
NAME_CONTRACT_STATUS_Active
mean
NAME_CONTRACT_STATUS_Approved
mean
NAME_CONTRACT_STATUS_Completed
mean
NAME_CONTRACT_STATUS_Demand
mean
NAME_CONTRACT_STATUS_Refused
mean
NAME_CONTRACT_STATUS_Sent proposal
mean
NAME_CONTRACT_STATUS_Signed
sum
SK_ID_PREV
sum
MONTHS_BALANCE
sum
AMT_BALANCE
sum
AMT_CREDIT_LIMIT_ACTUAL
sum
AMT_DRAWINGS_ATM_CURRENT
sum
AMT_DRAWINGS_CURRENT
sum
AMT_DRAWINGS_OTHER_CURRENT
sum
AMT_DRAWINGS_POS_CURRENT
sum
AMT_INST_MIN_REGULARITY

In [41]:
previous = pd.read_csv("input/previous_application.csv")

In [42]:
# Days 365.243 values -> nan
previous["DAYS_FIRST_DRAWING"].replace(365243, np.nan, inplace= True)
previous["DAYS_FIRST_DUE"].replace(365243, np.nan, inplace= True)
previous["DAYS_LAST_DUE_1ST_VERSION"].replace(365243, np.nan, inplace= True)
previous["DAYS_LAST_DUE"].replace(365243, np.nan, inplace= True)
previous["DAYS_TERMINATION"].replace(365243, np.nan, inplace= True)
# Add feature: value ask / value received percentage
previous["APP_CREDIT_PERC"] = previous["AMT_APPLICATION"] / previous["AMT_CREDIT"]

In [43]:
previous = les(previous)

In [44]:
train_test = groupbys(previous,train_test,"SK_ID_CURR")

mean
SK_ID_PREV
mean
AMT_ANNUITY
mean
AMT_APPLICATION
mean
AMT_CREDIT
mean
AMT_DOWN_PAYMENT
mean
AMT_GOODS_PRICE
mean
HOUR_APPR_PROCESS_START
mean
FLAG_LAST_APPL_PER_CONTRACT
mean
NFLAG_LAST_APPL_IN_DAY
mean
RATE_DOWN_PAYMENT
mean
RATE_INTEREST_PRIMARY
mean
RATE_INTEREST_PRIVILEGED
mean
DAYS_DECISION
mean
SELLERPLACE_AREA
mean
CNT_PAYMENT
mean
DAYS_FIRST_DRAWING
mean
DAYS_FIRST_DUE
mean
DAYS_LAST_DUE_1ST_VERSION
mean
DAYS_LAST_DUE
mean
DAYS_TERMINATION
mean
NFLAG_INSURED_ON_APPROVAL
mean
APP_CREDIT_PERC
mean
NAME_CONTRACT_TYPE_Cash loans
mean
NAME_CONTRACT_TYPE_Consumer loans
mean
NAME_CONTRACT_TYPE_Revolving loans
mean
NAME_CONTRACT_TYPE_XNA
mean
WEEKDAY_APPR_PROCESS_START_FRIDAY
mean
WEEKDAY_APPR_PROCESS_START_MONDAY
mean
WEEKDAY_APPR_PROCESS_START_SATURDAY
mean
WEEKDAY_APPR_PROCESS_START_SUNDAY
mean
WEEKDAY_APPR_PROCESS_START_THURSDAY
mean
WEEKDAY_APPR_PROCESS_START_TUESDAY
mean
WEEKDAY_APPR_PROCESS_START_WEDNESDAY
mean
NAME_CASH_LOAN_PURPOSE_Building a house or an annex
mean
NAME_C

In [48]:
# train = pd.read_csv("input/application_train.csv")
# test = pd.read_csv("input/application_test.csv")

In [56]:
train_test["SK_DPD_mean"].head()

0    NaN
1    NaN
2    NaN
3    0.0
4    NaN
Name: SK_DPD_mean, dtype: float64

In [52]:
# train_test = train.append(test)

ここからtrainデータの新しい特徴を増やしています。  
ここに関しては須崎さんのカーネルを拝見していただければ幸いです。

In [57]:
train_test['CODE_GENDER']=train_test['CODE_GENDER'].replace('XNA', np.nan, inplace=True)
train_test['DAYS_EMPLOYED']= train_test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
train_test['DAYS_LAST_PHONE_CHANGE'] = train_test['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True)
train_test['NAME_FAMILY_STATUS']= train_test['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
train_test['ORGANIZATION_TYPE'] = train_test['ORGANIZATION_TYPE'].replace('XNA', np.nan, inplace=True)


In [58]:
train_test["Cap_Adeq_Ratio"] = (train_test["AMT_INCOME_TOTAL"] + train_test["AMT_ANNUITY"] + train_test["AMT_GOODS_PRICE"] ) / train_test["AMT_CREDIT"]
train_test["Cap_Adeq_Ratio"].describe()
train_test["Acid_Ratio"] = train_test["AMT_CREDIT"] / train_test["AMT_INCOME_TOTAL"] 
train_test["Acid_Ratio"].describe()
train_test["Years_Employed"] = train_test["DAYS_EMPLOYED"] / -365
train_test["Years_Employed"].describe()
train_test["Years_Old"] = train_test["DAYS_BIRTH"] / -365
train_test["Years_Old"].describe()
train_test["Lifetime_employed_Ratio"] = train_test["Years_Employed"] / train_test["Years_Old"] 
train_test["Lifetime_employed_Ratio"].describe()
train_test["Age_of_Completion"] = round(train_test["Years_Old"] + (train_test["AMT_CREDIT"] / (train_test["AMT_INCOME_TOTAL"] /4)))
train_test["Age_of_Completion"].describe()
train_test["Period_of_Payment"] = round(train_test["Age_of_Completion"] - train_test["Years_Old"])
train_test["Period_of_Payment"].describe()
r = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.1]
g = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_test["REGION_POPULATION"] = pd.cut(train_test["REGION_POPULATION_RELATIVE"], bins=r, labels=g)
train_test["REGION_POPULATION"].describe()
train_test["NEW_CREDIT_TO_ANNUITY_RATIO"] = train_test["AMT_CREDIT"] / train_test["AMT_ANNUITY"]
train_test["NEW_CREDIT_TO_GOODS_RATIO"] = train_test["AMT_CREDIT"] / train_test["AMT_GOODS_PRICE"]
train_test["NEW_INC_PER_CHLD"] = train_test["AMT_INCOME_TOTAL"] / (1 + train_test["CNT_CHILDREN"])
train_test["NEW_EMPLOY_TO_BIRTH_RATIO"] = train_test["DAYS_EMPLOYED"] / train_test["DAYS_BIRTH"]
train_test["NEW_ANNUITY_TO_INCOME_RATIO"] = train_test["AMT_ANNUITY"] / (1 + train_test["AMT_INCOME_TOTAL"])
train_test["NEW_SOURCES_PROD"] = train_test["EXT_SOURCE_1"] * train_test["EXT_SOURCE_2"] * train_test["EXT_SOURCE_3"]
train_test["NEW_EXT_SOURCES_MEAN"] = train_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis=1)
train_test["NEW_SCORES_STD"] = train_test[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis=1)
train_test["NEW_SCORES_STD"] = train_test["NEW_SCORES_STD"].fillna(train_test["NEW_SCORES_STD"].mean())
train_test["NEW_CAR_TO_BIRTH_RATIO"] = train_test["OWN_CAR_AGE"] / train_test["DAYS_BIRTH"]
train_test["NEW_CAR_TO_EMPLOY_RATIO"] = train_test["OWN_CAR_AGE"] / train_test["DAYS_EMPLOYED"]
train_test["NEW_PHONE_TO_BIRTH_RATIO"] = train_test["DAYS_LAST_PHONE_CHANGE"] / train_test["DAYS_BIRTH"]
train_test["NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER"] = train_test["DAYS_LAST_PHONE_CHANGE"] / train_test["DAYS_EMPLOYED"]
train_test["NEW_CREDIT_TO_INCOME_RATIO"] = train_test["AMT_CREDIT"] / train_test["AMT_INCOME_TOTAL"]
train_test["external_sources_weighted"] = train_test.EXT_SOURCE_1 * 2 + train_test.EXT_SOURCE_2 * 3 + train_test.EXT_SOURCE_3 * 4

In [59]:
train_test.shape

(356255, 2536)

In [60]:
train_test.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,NEW_ANNUITY_TO_INCOME_RATIO,NEW_SOURCES_PROD,NEW_EXT_SOURCES_MEAN,NEW_SCORES_STD,NEW_CAR_TO_BIRTH_RATIO,NEW_CAR_TO_EMPLOY_RATIO,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER,NEW_CREDIT_TO_INCOME_RATIO,external_sources_weighted
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.121977,0.003043,0.161787,0.092026,NaN,NaN,NaN,NaN,2.007889,1.512423
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.132216,NaN,0.466757,0.219895,NaN,NaN,NaN,NaN,4.790750,NaN
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099999,NaN,0.642739,0.122792,-0.001365,NaN,NaN,NaN,2.000000,NaN
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.219898,NaN,0.650442,0.151008,NaN,NaN,NaN,NaN,2.316167,NaN
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179961,NaN,0.322738,0.151008,NaN,NaN,NaN,NaN,4.222222,NaN


In [60]:
train_test.to_csv("new_data/train_test.csv")